In [ ]:
import os
import json
import pickle
from openai import OpenAI

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [11]:
# Existing categories and secret words
with open("chameleon_cards_original.pkl", "rb") as f:
    chameleon_cards = pickle.load(f)

chameleon_cards_str = json.dumps(chameleon_cards, indent=2)
print(chameleon_cards_str)

In [32]:
# Collect new cards
new_cards = dict()
all_interactions = []
errs = []
num_new_cards = 60
num_new_cards_batch = 60
gen_kwargs = {
    "model": "gpt-4o-2024-08-06",
    "temperature": 0.8,
}

for i in range(0, num_new_cards, num_new_cards_batch):
    print(f"Generating new cards {i} to {i + num_new_cards_batch}. Number of new cards so far: {len(new_cards)}")
    
    # Construct the prompt
    prompt_num_new_cards = min(num_new_cards_batch, num_new_cards - i)    
    prompt = """
We are playing the game Chameleon but do not have enough game cards (categories and possible secret words).
Please suggest {n_new_cards} new categories and associated 16 secret words for the game. Here are the existing categories and secret words:

{chameleon_cards_str}

Please provide the {n_new_cards} new categories and secret words in a JSON format, where each category is a key and the value is a list of secret words. Do not say anything else, just return the JSON.{additional_instruction}
    """.strip().format(n_new_cards=prompt_num_new_cards, chameleon_cards_str=chameleon_cards_str, additional_instruction="")

    # Call the OpenAI API to get new categories and secret words
    response = client.responses.create(input=prompt, **gen_kwargs)
    all_interactions.append((prompt, response.output_text))

    # Extract the new cards from the response
    try:
        current_new_cards = json.loads(response.output_text.replace("```json", "").replace("```", "").strip())
        new_cards.update(current_new_cards)
    except json.JSONDecodeError as e:
        print("Failed to decode JSON response:", e)
        print("Response text:", response.output_text)
        errs.append((prompt, response.output_text))
    except Exception as e:
        print("An error occurred:", e)
        errs.append((prompt, response.output_text))

In [ ]:
# Save the results
import datetime
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")

with open(f"new_chameleon_cards_{timestamp}.pkl", "wb") as f:
    pickle.dump(new_cards, f)
with open(f"new_chameleon_cards_detail_{timestamp}.pkl", "wb") as f:
    pickle.dump({
        "new_cards": new_cards,
        "all_interactions": all_interactions,
        "errors": errs,
        "gen_kwargs": gen_kwargs,
    }, f)

print(f"Generated {len(new_cards)} new cards.")
print(f"Saved new cards to new_chameleon_cards_{timestamp}.pkl and new_chameleon_cards_detail_{timestamp}.pkl")

## Combine

In [23]:
all_cards = {**chameleon_cards, **new_cards}
print(f"Number of combined cards: {len(all_cards)}")

# Save the combined cards
with open("chameleon_cards.pkl", "wb") as f:
    pickle.dump(all_cards, f)